## 1. Setup & Configuration

In [6]:
import csv, pathlib, requests, time
from urllib.parse import urlparse

csv_path = pathlib.Path('./links_table.csv')
new_csv_path = pathlib.Path('./links_table_with_status.csv')

Imports required libraries and defines file paths for input CSV and output files.

## 2. Load Input Data

In [7]:
rows = []
with csv_path.open(newline='', encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for row in reader:
        rows.append(row)

Reads the CSV file containing links into a list of dictionaries.

## 3. HTTP Session Setup

In [8]:
session = requests.Session()
session.headers.update({'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'})

Creates a reusable HTTP session with a custom user agent.

## 4. Link Validation Loop

In [9]:
results = []
for i, row in enumerate(rows, 1):
    url = row['Link URL']
    alive = 'No'
    try:
        resp = session.head(url, allow_redirects=True, timeout=10)
        if resp.status_code < 400:
            alive = 'Yes'
        else:
            resp = session.get(url, stream=True, allow_redirects=True, timeout=15)
            if resp.status_code < 400:
                alive = 'Yes'
    except Exception:
        try:
            resp = session.get(url, stream=True, allow_redirects=True, timeout=15)
            if resp.status_code < 400:
                alive = 'Yes'
        except Exception:
            alive = 'No'
    
    results.append({
        'Source Page': row['Source Page'],
        'Link Text': row['Link Text'],
        'Link URL': url,
        'Alive': alive
    })
    
    if i % 50 == 0:
        time.sleep(1)

Checks each URL using HEAD request first, falls back to GET if needed, and records whether the link is alive.

## 5. Write CSV Output

In [10]:
with new_csv_path.open('w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['Source Page', 'Link Text', 'Link URL', 'Alive'])
    writer.writeheader()
    writer.writerows(results)

print('CSV saved to', new_csv_path)

CSV saved to links_table_with_status.csv
